<h1>Jobs analysis project</h1>
This is a project where I built a data pipeline, continuously collecting new data on data science, engineering, and analytics jobs from monster.com, then perform analyses and visualizations on how various aspects of the collected data affect salaries in the field. The focus of the project is on the pipeline itself, with the other aspects primarily being there to give the pipeline something to do. I used Julia for the entire pipeline. I used the Gumbo and Cascadia Julia libraries for web scraping, and collected and organized the data into a SQLite database.